# Bibliotecas necesarias

In [1]:
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
import AC
# Ignore DtypeWarnings from pandas' read_csv                                                                                                                                                                                            
warnings.filterwarnings('ignore', message="^Columns.*")

# Funciones necesarias

In [2]:
def frameToDict(path, keyword='CLAVE'):
    # Get a dictionary code:value from the csv
    dict_temp = (
        pd.read_csv(path)
        .set_index(keyword)
        .squeeze(axis=1)
        .to_dict()
    )
    return dict_temp

# Se carga el conjunto de datos

In [3]:
data = pd.read_csv('Archivos/220507COVID19MEXICO.csv')

In [4]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2022-05-07,z45dcb,2,12,9,2,9,9,3,1,...,1,1,2,2,97,7,99,México,97,97
1,2022-05-07,z477ab,1,12,12,2,12,12,59,2,...,2,2,97,2,97,6,99,México,97,2
2,2022-05-07,zz5293,1,12,29,1,29,29,5,1,...,2,1,2,2,97,7,99,México,97,97
3,2022-05-07,z1d9a5,1,12,22,2,9,22,14,1,...,2,1,1,2,97,3,99,México,97,97
4,2022-05-07,z1da2d,1,12,2,1,2,2,4,2,...,99,2,97,2,97,6,99,México,97,2


# Se cargan los archivos para realizar el mapeo

In [5]:
origen = frameToDict('Archivos/origen.csv')
sector = frameToDict('Archivos/sector.csv')
entidades = frameToDict('Archivos/entidades.csv')
sexo = frameToDict('Archivos/sexo.csv')
municipios = frameToDict('Archivos/municipios.csv')
paciente = frameToDict('Archivos/paciente.csv')
nacionalidad = frameToDict('Archivos/nacionalidad.csv')
si_no = frameToDict('Archivos/si-no.csv')
resultado_laboratorio = frameToDict('Archivos/resultado_laboratorio.csv')
resultados_antigenos = frameToDict('Archivos/resultado_antigenos.csv')

# Se replazan todas las columnas

In [6]:
data.ORIGEN = data.ORIGEN.replace(origen)
data.SECTOR = data.SECTOR.replace(sector)
data.ENTIDAD_UM = data.ENTIDAD_UM.replace(entidades)
data.SEXO = data.SEXO.replace(sexo)
data.ENTIDAD_NAC = data.ENTIDAD_NAC.replace(entidades)
data.ENTIDAD_RES = data.ENTIDAD_RES.replace(entidades)
data.MUNICIPIO_RES = data.MUNICIPIO_RES.replace(municipios)
data.TIPO_PACIENTE = data.TIPO_PACIENTE.replace(paciente)
data.NEUMONIA = data.NEUMONIA.replace(si_no)
data.NACIONALIDAD = data.NACIONALIDAD.replace(nacionalidad)
data.EMBARAZO = data.EMBARAZO.replace(si_no)
data.HABLA_LENGUA_INDIG = data.HABLA_LENGUA_INDIG.replace(si_no)
data.INDIGENA = data.INDIGENA.replace(si_no)
data.DIABETES = data.DIABETES.replace(si_no)
data.EPOC = data.EPOC.replace(si_no)
data.ASMA = data.ASMA.replace(si_no)
data.INMUSUPR = data.INMUSUPR.replace(si_no)
data.HIPERTENSION = data.HIPERTENSION.replace(si_no)
data.OTRA_COM = data.OTRA_COM.replace(si_no)
data.CARDIOVASCULAR = data.CARDIOVASCULAR.replace(si_no)
data.OBESIDAD = data.OBESIDAD.replace(si_no)
data.RENAL_CRONICA = data.RENAL_CRONICA.replace(si_no)
data.TABAQUISMO = data.TABAQUISMO.replace(si_no)
data.OTRO_CASO = data.OTRO_CASO.replace(si_no)
data.TOMA_MUESTRA_LAB = data.TOMA_MUESTRA_LAB.replace(si_no)
data.TOMA_MUESTRA_ANTIGENO = data.TOMA_MUESTRA_ANTIGENO.replace(si_no)
data.MIGRANTE = data.MIGRANTE.replace(si_no)
data.UCI = data.UCI.replace(si_no)
data.FECHA_INGRESO = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%dT')
data.FECHA_SINTOMAS = pd.to_datetime(data.FECHA_SINTOMAS, format='%Y-%m-%dT')
data.INTUBADO = data.INTUBADO.replace(si_no)
data.RESULTADO_LAB = data.RESULTADO_LAB.replace(resultado_laboratorio)
data.RESULTADO_ANTIGENO = data.RESULTADO_ANTIGENO.replace(resultados_antigenos)

In [7]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2022-05-07,z45dcb,FUERA DE USMER,SSA,CIUDAD DE MÉXICO,HOMBRE,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,ATOLINGA,AMBULATORIO,...,SI,SI,NO POSITIVO A SARS-COV-2,NO,NO APLICA (CASO SIN MUESTRA),7,NO ESPECIFICADO,México,97,NO APLICA
1,2022-05-07,z477ab,USMER,SSA,GUERRERO,HOMBRE,GUERRERO,GUERRERO,PROGRESO,HOSPITALIZADO,...,NO,NO,NO APLICA (CASO SIN MUESTRA),NO,NO APLICA (CASO SIN MUESTRA),6,NO ESPECIFICADO,México,97,NO
2,2022-05-07,zz5293,USMER,SSA,TLAXCALA,MUJER,TLAXCALA,TLAXCALA,CALERA,AMBULATORIO,...,NO,SI,NO POSITIVO A SARS-COV-2,NO,NO APLICA (CASO SIN MUESTRA),7,NO ESPECIFICADO,México,97,NO APLICA
3,2022-05-07,z1d9a5,USMER,SSA,QUERÉTARO,HOMBRE,CIUDAD DE MÉXICO,QUERÉTARO,GENERAL FRANCISCO R. MURGUÍA,AMBULATORIO,...,NO,SI,POSITIVO A SARS-COV-2,NO,NO APLICA (CASO SIN MUESTRA),3,NO ESPECIFICADO,México,97,NO APLICA
4,2022-05-07,z1da2d,USMER,SSA,BAJA CALIFORNIA,MUJER,BAJA CALIFORNIA,BAJA CALIFORNIA,BENITO JUÁREZ,HOSPITALIZADO,...,NO ESPECIFICADO,NO,NO APLICA (CASO SIN MUESTRA),NO,NO APLICA (CASO SIN MUESTRA),6,NO ESPECIFICADO,México,97,NO


# Se verifica que no existan valores nulos

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15855490 entries, 0 to 15855489
Data columns (total 40 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   FECHA_ACTUALIZACION    object        
 1   ID_REGISTRO            object        
 2   ORIGEN                 object        
 3   SECTOR                 object        
 4   ENTIDAD_UM             object        
 5   SEXO                   object        
 6   ENTIDAD_NAC            object        
 7   ENTIDAD_RES            object        
 8   MUNICIPIO_RES          object        
 9   TIPO_PACIENTE          object        
 10  FECHA_INGRESO          datetime64[ns]
 11  FECHA_SINTOMAS         datetime64[ns]
 12  FECHA_DEF              object        
 13  INTUBADO               object        
 14  NEUMONIA               object        
 15  EDAD                   int64         
 16  NACIONALIDAD           object        
 17  EMBARAZO               object        
 18  HABLA_LENGUA_INDIG  

# Se crean las etiquetas a utilizar

**Si el paciente es positivo a COVID**

In [9]:
target_name = 'CLASIFICACION_FINAL'
target = data[target_name]
data = data.drop(columns=[target_name])
# Si la clasificacion final es 1,2 o 3 el paciente fue positivo
data['CONFIRMADO'] = pd.Series(np.where(target <=3, 1, 0))

**Si el paciente fallecio**

In [10]:
target_name = 'FECHA_DEF'
target = data[target_name]
data = data.drop(columns=[target_name])
data['DEFUNCION'] = pd.Series(np.where(target!='9999-99-99',1,0))

# Preprocesamiento necesario

**Se arreglan datos**

In [11]:
data.PAIS_ORIGEN[(data.PAIS_ORIGEN == 97) | (data.PAIS_ORIGEN == '97')] = 'NO APLICA'

/var/folders/jp/10my5v9d6dl9nk7f_t7g56840000gn/T/ipykernel_8878/1697555608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.PAIS_ORIGEN[(data.PAIS_ORIGEN == 97) | (data.PAIS_ORIGEN == '97')] = 'NO APLICA'


**Se descartan las dos primeras columnas**

In [12]:
columns_to_drop = ['FECHA_ACTUALIZACION','ID_REGISTRO']
data = data.drop(columns=columns_to_drop)

**Se descartan las primeras dos columnas**

In [13]:
total = data.shape[0]
data = data.drop_duplicates()
print(f'Duplicated instances: {total - data.shape[0]}')

Duplicated instances: 674695


**Se pasan las fechas a numero ordinarios**

In [14]:
# Number of days since January 1 of year 1
data.FECHA_INGRESO = pd.to_datetime(data.FECHA_INGRESO).apply(pd.Timestamp.toordinal)
data.FECHA_SINTOMAS = pd.to_datetime(data.FECHA_SINTOMAS).apply(pd.Timestamp.toordinal)

In [15]:
data.FECHA_INGRESO = data.FECHA_INGRESO - data.FECHA_INGRESO.min()
data.FECHA_SINTOMAS = data.FECHA_SINTOMAS - data.FECHA_SINTOMAS.min()

In [16]:
data.head()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,...,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,CONFIRMADO,DEFUNCION
0,FUERA DE USMER,SSA,CIUDAD DE MÉXICO,HOMBRE,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,ATOLINGA,AMBULATORIO,230,228,...,SI,NO POSITIVO A SARS-COV-2,NO,NO APLICA (CASO SIN MUESTRA),NO ESPECIFICADO,México,NO APLICA,NO APLICA,0,0
1,USMER,SSA,GUERRERO,HOMBRE,GUERRERO,GUERRERO,PROGRESO,HOSPITALIZADO,153,147,...,NO,NO APLICA (CASO SIN MUESTRA),NO,NO APLICA (CASO SIN MUESTRA),NO ESPECIFICADO,México,NO APLICA,NO,0,0
2,USMER,SSA,TLAXCALA,MUJER,TLAXCALA,TLAXCALA,CALERA,AMBULATORIO,184,182,...,SI,NO POSITIVO A SARS-COV-2,NO,NO APLICA (CASO SIN MUESTRA),NO ESPECIFICADO,México,NO APLICA,NO APLICA,0,0
3,USMER,SSA,QUERÉTARO,HOMBRE,CIUDAD DE MÉXICO,QUERÉTARO,GENERAL FRANCISCO R. MURGUÍA,AMBULATORIO,342,337,...,SI,POSITIVO A SARS-COV-2,NO,NO APLICA (CASO SIN MUESTRA),NO ESPECIFICADO,México,NO APLICA,NO APLICA,1,0
4,USMER,SSA,BAJA CALIFORNIA,MUJER,BAJA CALIFORNIA,BAJA CALIFORNIA,BENITO JUÁREZ,HOSPITALIZADO,169,163,...,NO,NO APLICA (CASO SIN MUESTRA),NO,NO APLICA (CASO SIN MUESTRA),NO ESPECIFICADO,México,NO APLICA,NO,0,1


# Se guardan los datos

In [17]:
data.to_csv('all_atributes/covid_data_all_atributes.csv', index=False)

# Se crean los conjuntos de entrenamiento y pruebas

In [18]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [19]:
train_data.to_csv('all_atributes/all_atributes_train_data.csv', index=False)

In [20]:
test_data.to_csv('all_atributes/all_atributes_test_data.csv', index=False)